In [47]:
import skfuzzy as fuzz
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy.control as ctrl

In [118]:
class FuzzyControler():
    def __init__(self):
        self.forward = None
        self.left = None
        self.right = None
        self.output_left = None
        self.output_right = None
        self.rules = []
        self.control = None
        self.simulator = None
    
    def create_inputs(self):
        x_dst = np.arange(0, 1, 0.00001)
        
        self.forward = ctrl.Antecedent(x_dst, 'forward')
        self.left = ctrl.Antecedent(x_dst, 'left')
        self.right = ctrl.Antecedent(x_dst, 'right')
        
        self.forward['near'] = fuzz.trapmf(x_dst, [0.00397, 0.0145, 0.242063492063492, 0.374])
        self.forward['medium'] = fuzz.trapmf(x_dst, [0.295, 0.366, 0.55, 0.652116402116402])
        self.forward['far'] = fuzz.trapmf(x_dst, [0.583333333333333, 0.702, 1.00, 1.0])
        
        self.left['near'] = fuzz.trapmf(x_dst, [0.00132, 0.0119, 0.261, 0.358465608465608])
        self.left['medium'] = fuzz.trapmf(x_dst, [0.287, 0.374, 0.55, 0.675925925925926])
        self.left['far'] = fuzz.trapmf(x_dst, [0.61, 0.705026455026455, 1.03, 1.3])
        
        self.right['near'] = fuzz.trapmf(x_dst, [0.00132, 0.0119, 0.261, 0.358465608465608])
        self.right['medium'] = fuzz.trapmf(x_dst, [0.287, 0.374, 0.55, 0.675925925925926])
        self.right['far'] = fuzz.trapmf(x_dst, [0.61, 0.705026455026455, 1.03, 1.3])
        
        return
    
    def create_outputs(self):
        x_out = np.arange(-1,1, 0.00001)
        
        self.output_left = ctrl.Consequent(x_out, 'output_left')
        self.output_right = ctrl.Consequent(x_out, 'output_right')
        
        self.output_left['reverse'] = fuzz.trapmf(out_dst, [-1, -1, 0, 0])
        self.output_left['slow'] = fuzz.trapmf(out_dst, [0.0, 0.0, 0.5, 0.5])
        self.output_left['normal'] = fuzz.trapmf(out_dst, [0.4, 0.4, 0.8, 0.8])
        self.output_left['fast'] = fuzz.trapmf(out_dst, [0.7, 0.7, 1.0, 1.0])
        
        self.output_right['reverse'] = fuzz.trapmf(out_dst, [-1, -1, 0, 0])
        self.output_right['slow'] = fuzz.trapmf(out_dst, [0.0, 0.0, 0.5, 0.5])
        self.output_right['normal'] = fuzz.trapmf(out_dst, [0.4, 0.4, 0.8, 0.8])
        self.output_right['fast'] = fuzz.trapmf(out_dst, [0.7, 0.7, 1.0, 1.0])
        
        return
    
    def create_rules(self, forward, left, right, output_left, output_right):
        # rule 1
        rule1 = ctrl.Rule(antecedent=(forward['near'] & left['near'] & right['far']),
                          consequent=(output_left['fast'], output_right['slow']))
        # rule 2
        rule2 = ctrl.Rule(antecedent=(forward['near'] & left['near'] & right['medium']),
                          consequent=(output_left['normal'], output_right['fast']))
        # rule 3
        rule3 = ctrl.Rule(antecedent=(forward['medium'] & left['medium'] & right['far']),
                          consequent=(output_left['normal'], output_right['slow']))
        # rule 4
        rule4 = ctrl.Rule(antecedent=(forward['medium'] & left['near'] & right['far']),
                          consequent=(output_left['fast'], output_right['slow']))
        # rule 5
        rule5 = ctrl.Rule(antecedent=(forward['medium'] & left['medium'] & right['medium']),
                          consequent=(output_left['slow'], output_right['slow']))
        # rule 6
        rule6 = ctrl.Rule(antecedent=(forward['medium'] & left['near'] & right['medium']),
                          consequent=(output_left['fast'], output_right['slow']))
        # rule 7
        rule7 = ctrl.Rule(antecedent=(forward['far'] & left['far'] & right['far']),
                          consequent=(output_left['fast'], output_right['fast']))
        # rule 8
        rule8 = ctrl.Rule(antecedent=(forward['near'] & left['medium'] & right['far']),
                          consequent=(output_left['slow'], output_right['fast']))
        # rule 9
        rule9 = ctrl.Rule(antecedent=(forward['near'] & left['near']),
                          consequent=(output_left['slow'], output_right['fast']))
        # rule 10
        rule10 = ctrl.Rule(antecedent=(left['near']),
                          consequent=(output_left['fast'], output_right['slow']))
        # rule 11
        rule11 = ctrl.Rule(antecedent=(forward['medium'] & left['medium'] & right['near']),
                          consequent=(output_left['normal'], output_right['reverse']))
        # rule 12
        rule12 = ctrl.Rule(antecedent=(forward['medium'] & left['far'] & right['near']),
                          consequent=(output_left['slow'], output_right['fast']))
        # rule 13
        rule13 = ctrl.Rule(antecedent=(forward['near'] & left['medium'] & right['near']),
                          consequent=(output_left['fast'], output_right['slow']))
        # rule 14
        rule14 = ctrl.Rule(antecedent=(forward['near'] & right['near']),
                          consequent=(output_left['slow'], output_right['normal']))
        # rule 15
        rule15 = ctrl.Rule(antecedent=(right['near']),
                          consequent=(output_left['reverse'], output_right['normal']))
        # rule 16
        rule16 = ctrl.Rule(antecedent=(forward['near']),
                          consequent=(output_left['reverse'], output_right['normal']))
        
        for i in range(1, 17):
            self.rules.append(eval("rule" + str(i)))
            
        return
    
    def create_control(self):
        self.create_inputs()
        self.create_outputs()
        self.create_rules(self.forward, self.left, self.right, self.output_left, self.output_right)
        
        self.control = skfuzzy.control.ControlSystem(self.rules)
        
        return
    
    def show_fuzzy(self):
        if self.control is None:
            raise Exception("Control not created yet!")
        else:
            self.control.view()
    
        return
    
    def create_simulator(self):
        if self.control is None:
            self.create_control()
        
        self.simulator = ctrl.ControlSystemSimulation(self.control)
        
        return
            
    def simulate(self, input_foward=None, input_left=None, input_right=None):
        if self.simulator is None:
            self.create_simulator()
            
        if input_foward is None or input_left is None or input_right is None:
            raise Exception("Inputs can't be none")
        
        self.simulator.input['forward'] = input_foward
        self.simulator.input['left'] = input_left
        self.simulator.input['right'] = input_right
        
        self.simulator.compute()
        
        return self.simulator.output['output_left'], self.simulator.output['output_right']